In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from typing import Dict, Any
from sklearn.metrics import accuracy_score

In [31]:
cname = ['wrist x value', 'wrist y value', 'wrist z value', 'thigh x value', 'thigh y value', 'thigh z value', 'class']
cname2 = ['wrist x value', 'wrist y value', 'wrist z value', 'thigh x value', 'thigh y value', 'thigh z value']
train = pd.read_csv('training.csv', names=cname, header=None)
df_train = pd.DataFrame(train)
df_train

wrist x value  wrist y value  wrist z value  thigh x value  \
0          -0.453125      -1.234375       0.250000      -1.218750   
1          -0.453125      -1.218750       0.265625      -1.250000   
2          -0.468750      -1.187500       0.281250      -1.296875   
3          -0.468750      -1.109375       0.296875      -1.375000   
4          -0.468750      -1.031250       0.296875      -1.437500   
...              ...            ...            ...            ...   
29995      -0.468750      -0.859375       0.296875      -0.937500   
29996      -0.468750      -0.875000       0.296875      -0.953125   
29997      -0.453125      -0.875000       0.296875      -0.953125   
29998      -0.437500      -0.875000       0.296875      -0.953125   
29999      -0.437500      -0.875000       0.312500      -0.953125   

       thigh y value  thigh z value     class  
0          -0.343750       0.078125  walkfast  
1          -0.281250      -0.078125  walkfast  
2          -0.218750      -0.125000  walkfast  
3          -0.218750      -0.125000  walkfast  
4          -0.281250      -0.125000  walkfast  
...              ...            ...       ...  
29995      -0.171875       0.218750  walkslow  
29996      -0.187500       0.218750  walkslow  
29997      -0.203125       0.187500  walkslow  
29998      -0.203125       0.171875  walkslow  
29999      -0.187500       0.156250  walkslow  

[30000 rows x 7 columns]

In [32]:
test = pd.read_csv('testing.csv', names=cname2, header=None)
test

wrist x value wrist y value  wrist z value thigh x value thigh y value  \
0       -0.390625    -1.109.375       0.218750     -0.828125     -0.109375   
1       -0.390625      -109.375       0.203125       -0.8125       -0.1875   
2          -0.375       -10.625       0.187500       -0.8125     -0.265625   
3          -0.375    -1.046.875       0.171875       -0.8125     -0.265625   
4        -0.34375    -1.015.625       0.171875       -0.8125     -0.203125   
..            ...           ...            ...           ...           ...   
295     -0.484375     -0.859375       0.218750      -0.96875      -0.28125   
296          -0.5        -0.875       0.218750     -0.953125       -0.3125   
297          -0.5     -0.890625       0.218750      -0.96875       -0.3125   
298          -0.5      -0.90625       0.218750          -1.0      -0.28125   
299          -0.5     -0.921875       0.218750          -1.0         -0.25   

    thigh z value  
0         0.15625  
1         0.28125  
2         0.28125  
3         0.21875  
4          0.1875  
..            ...  
295        0.3125  
296      0.328125  
297       0.34375  
298      0.328125  
299      0.265625  

[300 rows x 6 columns]

In [33]:
# change the summary into numerical value
df_train['class'] = df_train['class'].map({'walkfast': 0, 'walkmod': -1, 'walkslow': 1})
df_train

wrist x value  wrist y value  wrist z value  thigh x value  \
0          -0.453125      -1.234375       0.250000      -1.218750   
1          -0.453125      -1.218750       0.265625      -1.250000   
2          -0.468750      -1.187500       0.281250      -1.296875   
3          -0.468750      -1.109375       0.296875      -1.375000   
4          -0.468750      -1.031250       0.296875      -1.437500   
...              ...            ...            ...            ...   
29995      -0.468750      -0.859375       0.296875      -0.937500   
29996      -0.468750      -0.875000       0.296875      -0.953125   
29997      -0.453125      -0.875000       0.296875      -0.953125   
29998      -0.437500      -0.875000       0.296875      -0.953125   
29999      -0.437500      -0.875000       0.312500      -0.953125   

       thigh y value  thigh z value  class  
0          -0.343750       0.078125      0  
1          -0.281250      -0.078125      0  
2          -0.218750      -0.125000      0  
3          -0.218750      -0.125000      0  
4          -0.281250      -0.125000      0  
...              ...            ...    ...  
29995      -0.171875       0.218750      1  
29996      -0.187500       0.218750      1  
29997      -0.203125       0.187500      1  
29998      -0.203125       0.171875      1  
29999      -0.187500       0.156250      1  

[30000 rows x 7 columns]

In [34]:
X_train = df_train.iloc[:, 0:6].values
y_train = df_train.iloc[:, 6].values
X_test = test.iloc[:, 0:6]
X_test = X_test.apply(pd.to_numeric, errors = 'coerce')
data = X_train

In [35]:
class SVM:
    def __init__(self, learning_rate=0.01, lambda_param=0.01, num_iterations=500):
        self.learning_rate = learning_rate
        self.lambda_param = lambda_param
        self.num_iterations = num_iterations
        self.weights = None
        self.bias = None
    
    def fit(self, X, y):
        num_samples, num_features = X.shape
        
        # Initialize weights and bias
        self.weights = np.zeros(num_features)
        self.bias = 0
        
        # Gradient descent
        for _ in range(self.num_iterations):
            # Calculate the hinge loss and gradients
            loss = 0
            dw = np.zeros(num_features)
            db = 0
            
            for i in range(num_samples):
                if y[i] * (np.dot(X[i], self.weights) - self.bias) >= 1:
                    dw += 0
                    db += 0
                else:
                    dw += self.lambda_param * y[i] * X[i]
                    db += self.lambda_param * y[i]
                    loss += 1 - y[i] * (np.dot(X[i], self.weights) - self.bias)
            
            # Update weights and bias
            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db
        
    def predict(self, X):
        y_pred = np.sign(np.dot(X, self.weights) - self.bias)
        return y_pred


In [36]:
clf = SVM()
clf.fit(np.array(X_train), np.array(y_train))
y_pred = clf.predict(np.array(X_test))
y_pred

array([nan,  1.,  1., nan, nan,  1.,  1.,  1.,  1., nan, nan,  1., nan,
       nan,  1.,  1., nan, nan, nan,  1., nan, nan, nan, nan, nan, nan,
        1.,  1.,  1.,  1., nan, nan,  1.,  1., nan,  1.,  1., nan,  1.,
       nan, nan,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1., nan,  1.,  1., nan,  1., nan, nan, nan,  1.,
       nan, nan, nan, nan,  1.,  1.,  1., nan, nan,  1.,  1.,  1.,  1.,
        1.,  1.,  1., nan, nan, nan, nan,  1., nan,  1.,  1., nan,  1.,
        1.,  1., nan,  1., nan, nan,  1., nan,  1., nan, nan,  1., nan,
        1.,  1., nan, nan, nan, nan,  1.,  1., nan, nan,  1.,  1.,  1.,
        1.,  1.,  1.,  1., nan,  1.,  1.,  1.,  1., nan, nan,  1., nan,
       nan, nan,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1., nan,  1.,  1.,  1., nan, nan, nan, nan,  1., nan,
       nan,  1., nan, nan, nan,  1., nan,  1., nan, nan,  1., na

In [53]:
id = np.arange(1, len(y_pred)+1)
# merge id with y_pred
y_pred = pd.DataFrame(y_pred)
y_pred = y_pred.apply(lambda x: x.map({0: 'walkfast', -1: 'walkmod', 1: 'walkslow'}))
y_pred
# export to csv with index name id and column name label
y_pred.to_csv('id_label.csv', index_label='id', header=['label'])